In [96]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [97]:
credits=pd.read_csv('credits.csv')
keywords=pd.read_csv('keywords.csv')
links=pd.read_csv('links_small.csv')
ratings=pd.read_csv('ratings_small.csv')

In [98]:
movies =pd.read_csv('movies_metadata.csv',low_memory=False)


In [99]:
movies.shape

(45466, 24)

In [100]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [101]:
movies.head(1)["production_companies"].values

array(["[{'name': 'Pixar Animation Studios', 'id': 3}]"], dtype=object)

In [102]:
movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [103]:
movies.duplicated().sum()

17

In [104]:
movies.drop_duplicates(inplace=True)

In [105]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45449 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45449 non-null  object 
 1   belongs_to_collection  4493 non-null   object 
 2   budget                 45449 non-null  object 
 3   genres                 45449 non-null  object 
 4   homepage               7780 non-null   object 
 5   id                     45449 non-null  object 
 6   imdb_id                45432 non-null  object 
 7   original_language      45438 non-null  object 
 8   original_title         45449 non-null  object 
 9   overview               44495 non-null  object 
 10  popularity             45444 non-null  object 
 11  poster_path            45063 non-null  object 
 12  production_companies   45446 non-null  object 
 13  production_countries   45446 non-null  object 
 14  release_date           45362 non-null  object 
 15  re

In [106]:
movies_cleaned_df=movies.drop(columns=["adult","belongs_to_collection","budget","homepage","imdb_id","release_date","runtime","video","status","poster_path","production_countries"])

In [107]:
movies_cleaned_df.head()

,genres,id,original_language,original_title,overview,popularity,production_companies,revenue,spoken_languages,tagline,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...",76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


In [108]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [109]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [110]:
movies_df_merged=pd.merge(keywords,credits,on="id")
movies_df_merged.dtypes

id           int64
keywords    object
cast        object
crew        object
dtype: object

In [111]:
movies["overview"]=movies["overview"].fillna(" ")
movies["overview"].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [112]:

movies = movies.drop([19730, 29503, 35587])
movies['id']=pd.to_numeric(movies['id'])
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45446 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45446 non-null  object 
 1   belongs_to_collection  4490 non-null   object 
 2   budget                 45446 non-null  object 
 3   genres                 45446 non-null  object 
 4   homepage               7777 non-null   object 
 5   id                     45446 non-null  int64  
 6   imdb_id                45429 non-null  object 
 7   original_language      45435 non-null  object 
 8   original_title         45446 non-null  object 
 9   overview               45446 non-null  object 
 10  popularity             45443 non-null  object 
 11  poster_path            45060 non-null  object 
 12  production_companies   45443 non-null  object 
 13  production_countries   45443 non-null  object 
 14  release_date           45359 non-null  object 
 15  re

In [113]:
movies_df_merged_full=pd.merge(movies_df_merged,movies,on="id")
movies_df_merged_full.head()

,id,keywords,cast,crew,adult,belongs_to_collection,budget,genres,homepage,imdb_id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,tt0113228,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0113041,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [114]:
movies_df_merged_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46548 entries, 0 to 46547
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     46548 non-null  int64  
 1   keywords               46548 non-null  object 
 2   cast                   46548 non-null  object 
 3   crew                   46548 non-null  object 
 4   adult                  46548 non-null  object 
 5   belongs_to_collection  4568 non-null   object 
 6   budget                 46548 non-null  object 
 7   genres                 46548 non-null  object 
 8   homepage               8001 non-null   object 
 9   imdb_id                46531 non-null  object 
 10  original_language      46537 non-null  object 
 11  original_title         46548 non-null  object 
 12  overview               46548 non-null  object 
 13  popularity             46544 non-null  object 
 14  poster_path            46149 non-null  object 
 15  pr

In [115]:
movies_df_merged_full.shape

(46548, 27)

In [116]:
movies_df_merged_full.isnull().sum()

id                           0
keywords                     0
cast                         0
crew                         0
adult                        0
belongs_to_collection    41980
budget                       0
genres                       0
homepage                 38547
imdb_id                     17
original_language           11
original_title               0
overview                     0
popularity                   4
poster_path                399
production_companies         4
production_countries         4
release_date                88
revenue                      4
runtime                    268
spoken_languages             4
status                      86
tagline                  25795
title                        4
video                        4
vote_average                 4
vote_count                   4
dtype: int64

In [117]:
#selecting most relevant features for recommendation

selected_features=["id","cast",'genres',"tagline","title","production_companies","overview","poster_path","popularity"]

In [118]:
selected_features

['id',
 'cast',
 'genres',
 'tagline',
 'title',
 'production_companies',
 'overview',
 'poster_path',
 'popularity']

In [119]:
movies_cleaned = movies_df_merged_full[selected_features]

In [120]:
#checking null values in selected datacolumns
movies_cleaned.isnull().sum()

id                          0
cast                        0
genres                      0
tagline                 25795
title                       4
production_companies        4
overview                    0
poster_path               399
popularity                  4
dtype: int64

In [121]:
#Checking the presence of duplicated data in dataframe

movies_cleaned=movies_cleaned.drop_duplicates()
movies_cleaned.duplicated().sum()

0

In [122]:
movies_cleaned.iloc[0].genres

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [123]:
#"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
# ['Animation','Comedy','Family']

In [124]:
#converting given string into python list and then extracting the genres
import ast
def convert(obj):
    list=[]
    #using ast.literal_eval(obj) to convert obj to list and then applying loop to get the genres(values at "name" index of genres)
    for i in ast.literal_eval(obj):
        list.append(i["name"])
    return list       

        

In [125]:
convert("[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]")


['Animation', 'Comedy', 'Family']

In [126]:
import ast
ast.literal_eval("[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]")

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [127]:
movies_cleaned["genres"]=movies_cleaned["genres"].apply(convert)

In [128]:
movies_cleaned[selected_features].head()

,id,cast,genres,tagline,title,production_companies,overview,poster_path,popularity
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[Animation, Comedy, Family]",NaN,Toy Story,"[{'name': 'Pixar Animation Studios', 'id': 3}]","Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,21.946943
1,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,Jumanji,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,17.015539
2,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,11.7129
3,31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,Waiting to Exhale,[{'name': 'Twentieth Century Fox Film Corporat...,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,3.859495
4,11862,"[{'cast_id': 1, 'character': 'George Banks', '...",[Comedy],Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,"[{'name': 'Sandollar Productions', 'id': 5842}...",Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,8.387519


In [129]:
selected_features

['id',
 'cast',
 'genres',
 'tagline',
 'title',
 'production_companies',
 'overview',
 'poster_path',
 'popularity']

In [130]:
movies_cleaned.drop(['production_companies'],axis=1,inplace=True)

In [131]:
movies_cleaned["tagline"].fillna(" ",inplace=True)

In [132]:
movies_cleaned.isnull().sum()

id               0
cast             0
genres           0
tagline          0
title            3
overview         0
poster_path    386
popularity       3
dtype: int64

In [133]:
#Removing rows with title with null values

movies_cleaned["title"]=movies_cleaned["title"].dropna()

In [134]:
movies_cleaned["cast"].head()

0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

In [135]:
movies_cleaned.isnull().sum()

id               0
cast             0
genres           0
tagline          0
title            3
overview         0
poster_path    386
popularity       3
dtype: int64

In [136]:
movies_cleaned.dropna(subset=["cast","popularity"],inplace=True)

In [137]:
movies_cleaned["cast"].head()

0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

In [138]:
movies_cleaned["cast"]=movies_cleaned["cast"].apply(convert)

In [139]:
# movies_cleaned=movies_cleaned[selected_features]
movies_cleaned.head()

,id,cast,genres,tagline,title,overview,poster_path,popularity
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Animation, Comedy, Family]",,Toy Story,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,21.946943
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,Jumanji,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,17.015539
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,11.7129
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,3.859495
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Comedy],Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,8.387519


In [140]:
movies_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45442 entries, 0 to 46547
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           45442 non-null  int64 
 1   cast         45442 non-null  object
 2   genres       45442 non-null  object
 3   tagline      45442 non-null  object
 4   title        45442 non-null  object
 5   overview     45442 non-null  object
 6   poster_path  45059 non-null  object
 7   popularity   45442 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.1+ MB


In [141]:
movies_cleaned.dropna()

,id,cast,genres,tagline,title,overview,poster_path,popularity
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Animation, Comedy, Family]",,Toy Story,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,21.946943
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,Jumanji,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,17.015539
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,11.7129
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,3.859495
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Comedy],Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,8.387519
...,...,...,...,...,...,...,...,...
46543,439050,"[Leila Hatami, Kourosh Tahami, Elham Korda]","[Drama, Family]",Rising and falling between a man and woman,Subdue,Rising and falling between a man and woman.,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,0.072051
46544,111109,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Drama],,Century of Birthing,An artist struggles to finish his work while a...,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,0.178241
46545,67758,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...","[Action, Drama, Thriller]",A deadly game of wits.,Betrayal,"When one of her hits goes wrong, a professiona...",/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,0.903007
46546,227506,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[],,Satan Triumphant,"In a small town live two brothers, one a minis...",/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,0.003503


In [142]:
#filling null values with null string

movies_cleaned["title"].fillna(" ",inplace=True)
movies_cleaned["tagline"].fillna(" ",inplace=True)
movies_cleaned.dropna(subset=["poster_path","cast"],inplace=True)

In [143]:
movies_cleaned.isnull().sum()

id             0
cast           0
genres         0
tagline        0
title          0
overview       0
poster_path    0
popularity     0
dtype: int64

In [144]:
movies_cleaned.head()

,id,cast,genres,tagline,title,overview,poster_path,popularity
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Animation, Comedy, Family]",,Toy Story,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,21.946943
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,Jumanji,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,17.015539
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,11.7129
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,3.859495
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Comedy],Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,8.387519


In [145]:
movies_cleaned["cast"].iloc[0]

['Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'Wallace Shawn',
 'John Ratzenberger',
 'Annie Potts',
 'John Morris',
 'Erik von Detten',
 'Laurie Metcalf',
 'R. Lee Ermey',
 'Sarah Freeman',
 'Penn Jillette']

In [146]:
# movies_cleaned["production_companies"].apply(convert)

In [147]:
movies_cleaned.head()

,id,cast,genres,tagline,title,overview,poster_path,popularity
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Animation, Comedy, Family]",,Toy Story,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,21.946943
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,Jumanji,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,17.015539
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,11.7129
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,3.859495
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Comedy],Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,8.387519


In [148]:
#converting overview column from string to list to concact all the required columns together
movies_cleaned["overview"]=movies_cleaned["overview"].apply(lambda x:x.split())
movies_cleaned["tagline"]=movies_cleaned["tagline"].apply(lambda x:x.split())

In [149]:
movies_cleaned["tagline"].head()

0                                                   []
1    [Roll, the, dice, and, unleash, the, excitement!]
2    [Still, Yelling., Still, Fighting., Still, Rea...
3    [Friends, are, the, people, who, let, you, be,...
4    [Just, When, His, World, Is, Back, To, Normal....
Name: tagline, dtype: object

In [150]:
#removing spaces between names in cast and genres example:Tom Hanks->TomHanks and Science Fiction->ScienceFiction
movies_cleaned["cast"]=movies_cleaned["cast"].apply(lambda x:[i.replace(" ","") for i in x])
movies_cleaned["genres"]=movies_cleaned["genres"].apply(lambda x:[i.replace(" ","") for i in x])

In [151]:
movies_cleaned["overview"].head()

0    [Led, by, Woody,, Andy's, toys, live, happily,...
1    [When, siblings, Judy, and, Peter, discover, a...
2    [A, family, wedding, reignites, the, ancient, ...
3    [Cheated, on,, mistreated, and, stepped, on,, ...
4    [Just, when, George, Banks, has, recovered, fr...
Name: overview, dtype: object

In [152]:
movies_cleaned=movies_cleaned.sort_values(by="popularity",ascending=False).head(10000)

In [153]:
#combining all the selected features
movies_cleaned["tags"]=movies_cleaned["cast"]+movies_cleaned["overview"]+movies_cleaned["genres"]+movies_cleaned["tagline"]



In [154]:
movies_cleaned.head()

,id,cast,genres,tagline,title,overview,poster_path,popularity,tags
34448,96159,"[AnilKapoor, AmritaSingh, AmjadKhan, OmPrakash...","[Comedy, Drama, Romance]",[],Chameli Ki Shaadi,"[Charandas,, a, young, man, in, a, village, ha...",/jHodRJMbZ7tpEVXZB2tw3achnXq.jpg,9e-06,"[AnilKapoor, AmritaSingh, AmjadKhan, OmPrakash..."
39665,85013,"[RickyPaullGoldin, DennisHolahan, SydneyPenny,...",[ScienceFiction],"[She, was, from, another, star., He, had, wait...",Hyper Sapien: People from Another Star,"[Three, aliens, from, the, planet, Taros, land...",/kFI5AnjWK124ILZXN5QMqLhbukl.jpg,9e-06,"[RickyPaullGoldin, DennisHolahan, SydneyPenny,..."
536,78,"[HarrisonFord, RutgerHauer, SeanYoung, EdwardJ...","[ScienceFiction, Drama, Thriller]","[Man, has, made, his, match..., now, it's, his...",Blade Runner,"[In, the, smog-choked, dystopian, Los, Angeles...",/p64TtbZGCElxQHpAMWmDHkWJlH2.jpg,96.272374,"[HarrisonFord, RutgerHauer, SeanYoung, EdwardJ..."
3516,10567,"[D.B.Sweeney, AlfreWoodard, OssieDavis, MaxCas...","[Animation, Family]","[You, have, never, seen, anything, like, this.]",Dinosaur,"[An, orphaned, dinosaur, raised, by, lemurs, j...",/jZDp3C5howi40aCeqXDJliodXk9.jpg,9.998415,"[D.B.Sweeney, AlfreWoodard, OssieDavis, MaxCas..."
10600,25,"[JamieFoxx, JakeGyllenhaal, ScottMacDonald, Lu...","[Drama, War]","[Welcome, to, the, suck.]",Jarhead,"[Jarhead, is, a, film, about, a, US, Marine, A...",/kmbyO0XUHRHcMyxVSZAWDdrpxIu.jpg,9.997032,"[JamieFoxx, JakeGyllenhaal, ScottMacDonald, Lu..."


In [155]:
movies_cleaned["tags"].iloc[0]

['AnilKapoor',
 'AmritaSingh',
 'AmjadKhan',
 'OmPrakash',
 'BharatiAchrekar',
 'AnnuKapoor',
 'RamSethi',
 'JayshreeT.',
 'ShailChaturvedi',
 'Rupini',
 'PankajKapur',
 'SatyendraKapoor',
 'Charandas,',
 'a',
 'young',
 'man',
 'in',
 'a',
 'village',
 'has',
 'no',
 'aim',
 'in',
 'life',
 'except',
 'to',
 'be',
 'a',
 'pehelwaan',
 '(wrestler).',
 'He',
 'takes',
 'an',
 'oath',
 'that',
 'he',
 'will',
 'not',
 'court',
 'or',
 'marry',
 'a',
 'girl',
 'until',
 'he',
 'hits',
 'the',
 'age',
 'of',
 'forty.',
 'But',
 'things',
 'changes',
 'when',
 'Charandas',
 'meets',
 'Chameli',
 'and',
 'falls',
 'in',
 'love',
 'with',
 'her.',
 'A',
 'hilarious',
 'take',
 'on',
 'the',
 'caste',
 'system',
 'of',
 'India,',
 'as',
 'both',
 'of',
 'them',
 'come',
 'from',
 'different',
 'castes',
 'and',
 'try',
 'hard',
 'to',
 'marry',
 'each',
 'other.',
 'Comedy',
 'Drama',
 'Romance']

In [156]:
new_df=movies_cleaned[["id","title","tags"]]
new_df.head()

,id,title,tags
34448,96159,Chameli Ki Shaadi,"[AnilKapoor, AmritaSingh, AmjadKhan, OmPrakash..."
39665,85013,Hyper Sapien: People from Another Star,"[RickyPaullGoldin, DennisHolahan, SydneyPenny,..."
536,78,Blade Runner,"[HarrisonFord, RutgerHauer, SeanYoung, EdwardJ..."
3516,10567,Dinosaur,"[D.B.Sweeney, AlfreWoodard, OssieDavis, MaxCas..."
10600,25,Jarhead,"[JamieFoxx, JakeGyllenhaal, ScottMacDonald, Lu..."


In [157]:
new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_41092\1804341217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))


In [158]:
new_df.head()

,id,title,tags
34448,96159,Chameli Ki Shaadi,AnilKapoor AmritaSingh AmjadKhan OmPrakash Bha...
39665,85013,Hyper Sapien: People from Another Star,RickyPaullGoldin DennisHolahan SydneyPenny Ros...
536,78,Blade Runner,HarrisonFord RutgerHauer SeanYoung EdwardJames...
3516,10567,Dinosaur,D.B.Sweeney AlfreWoodard OssieDavis MaxCasella...
10600,25,Jarhead,JamieFoxx JakeGyllenhaal ScottMacDonald LucasB...


In [159]:
new_df['tags'].iloc[0]

'AnilKapoor AmritaSingh AmjadKhan OmPrakash BharatiAchrekar AnnuKapoor RamSethi JayshreeT. ShailChaturvedi Rupini PankajKapur SatyendraKapoor Charandas, a young man in a village has no aim in life except to be a pehelwaan (wrestler). He takes an oath that he will not court or marry a girl until he hits the age of forty. But things changes when Charandas meets Chameli and falls in love with her. A hilarious take on the caste system of India, as both of them come from different castes and try hard to marry each other. Comedy Drama Romance'

In [160]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_41092\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [163]:
#Importing and using nltk for reducing the terms, in the document-term matrix, to its root term for the simplification of data

import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [196]:
ps.stem("engine running")

'engine run'

In [164]:
# Creating a helper fuction for doing the root term replacement
def stemmer (text):
    y=[] 
    
    for word in text.split(): #converting text to list using text.split for easy iteration
        y.append(ps.stem(word))  #storing the root word for the string in the list y using append
        
    return " ".join(y)

In [165]:
new_df['tags']=new_df['tags'].apply(stemmer)

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_41092\797096533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stemmer)


In [166]:
#importing sklearn for finding relationship between the movies using their associated texts

from sklearn.feature_extraction.text import CountVectorizer
cv =  CountVectorizer (max_features=5000,stop_words='english')

In [167]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [168]:
vectors.shape

(10000, 5000)

In [169]:
cv.get_feature_names()

C:\Users\ROHAN\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '18th',
 '19',
 '1920',
 '1930',
 '1940',
 '1941',
 '1942',
 '1944',
 '1945',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1963',
 '1970',
 '1970s',
 '1974',
 '1980',
 '1980s',
 '1986',
 '19th',
 '20',
 '200',
 '2009',
 '20th',
 '21st',
 '24',
 '25',
 '30',
 '300',
 '35',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 '80',
 '90',
 'aamirkhan',
 'aaron',
 'aaroneckhart',
 'aasifmandvi',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abroad',
 'absolut',
 'abus',
 'abuse',
 'academi',
 'academy',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'acquir',
 'act',
 'action',
 'activ',
 'activist',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adambrodi',
 'adamlefevr',
 'adamsandl',
 'adamscott',
 'adapt',
 'add',
 'addict',
 'adewaleakinnuoye',
 'adjust',
 'admir',
 'a

In [170]:
#importing cosine_similarity to calculate the similartiy between the text vectors we created

from sklearn.metrics.pairwise import cosine_similarity


In [171]:
similarity=cosine_similarity(vectors)

In [172]:
similarity[1]

array([0.07537784, 1.        , 0.22975187, ..., 0.10112998, 0.16256402,
       0.        ])

In [173]:
stemmer("tomhanks timallen donrickles led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences. animation comedy family")

"tomhank timallen donrickl led by woody, andy' toy live happili in hi room until andy' birthday bring buzz lightyear onto the scene. afraid of lose hi place in andy' heart, woodi plot against buzz. but when circumst separ buzz and woodi from their owner, the duo eventu learn to put asid their differences. anim comedi famili"

In [174]:
new_df[new_df["title"]=="Hamlet"].iloc[0].id

10549

In [175]:
sorted(list(enumerate(similarity[1])),reverse=True,key=lambda x:x[1])[0:5]

[(1, 0.9999999999999996),
 (9543, 0.39955629492721617),
 (9535, 0.3619613829965133),
 (9224, 0.3413943709994594),
 (3046, 0.3411211461689767)]

In [176]:
def recommend (movie):
    movie_index=new_df[new_df["title"]==movie].index[0]
    distance=similarity[movie_index]
    movies_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[0:5]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [177]:
similarity[1]

array([0.07537784, 1.        , 0.22975187, ..., 0.10112998, 0.16256402,
       0.        ])

In [178]:
new_df.iloc[7].title

'Hamlet'

In [179]:
movie_index=new_df[new_df["title"]=="Dinosaur"].index[0]
print(movie_index)

3516


In [180]:
new_df[new_df.title=='The Avengers']

,id,title,tags
2060,9320,The Avengers,ralphfienn umathurman seanconneri patrickmacne...
17944,24428,The Avengers,robertdowneyjr. chrisevan markruffalo chrishem...


In [181]:
#changing the indices of the data frame which had changed after sorting the dataframe with respect to the popularity of movies(done to reduce the size of the data frame)

new_df.index=range(len(new_df))

In [182]:
new_df.iloc[9990].tags

'johnnydepp orlandobloom keiraknightley stellanskarsgård billnighi jackdavenport kevinmcn jonathanpryc leearenberg mackenziecrook tomholland naomieharri davidbaili martinklebba davidschofield alexnorton geoffreyrush dermotkeaney andybeckwith cliveashborn reggiele christopheradamson jonathanlinsley johnboswal maxbak stevespeir laurenmah vanessabranch lukedewoolfson jonathankit ho-kwants peterdonaldbadalamentiii christophers.capp philipettington captain jack sparrow work hi way out of a blood debt with the ghostli davey jones, he also attempt to avoid etern damnation. adventur fantasi action jack is back!'

In [183]:
recommend("Hamlet")

Hamlet
Sword of Vengeance
Uncle Boonmee Who Can Recall His Past Lives
Elena
Two Queens and One Consort


In [184]:
recommend("Dinosaur")

Dinosaur
Grande, grosso e Verdone
The Land Before Time VIII: The Big Freeze
My Life as a Zucchini
Stick Man


In [198]:
movie=input("Enter Movie: ")

Enter Movie: Transformers


In [199]:
recommend(movie)

Transformers
Transformers: Revenge of the Fallen
Transformers: Dark of the Moon
Transformers: The Last Knight
Amira & Sam


In [187]:
import pickle

In [188]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [189]:
pickle.dump(similarity,open('similarity.pkl','wb'))